## This is the script to check the prepared data set in IPython 




In [78]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
from StringIO import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operationsb
from zipfile import ZipFile
import json
import requests

define your working dir!

In [79]:
cd 'C:\Users\Noam\Documents\Sync\DeepESA\wikilink_nc\'

C:\Users\Noam\Documents\Sync\DeepESA\wikilink_nc


In [80]:
def getZIP(zipFileName):
#     r = requests.get(zipFileName).content
#     s = StringIO.StringIO(r)
    zf = ZipFile(zipFileName, 'r') # Read in a list of zipped files
    return zf

In [81]:
zf = getZIP('wikilink_nc.zip')
print zf.namelist()

['wiki_test.json', 'wiki_test_2.json', 'README.txt', 'wikilink_0.json', 'wikilink_1.json', 'wikilink_10.json', 'wikilink_11.json', 'wikilink_12.json', 'wikilink_13.json', 'wikilink_14.json', 'wikilink_15.json', 'wikilink_16.json', 'wikilink_17.json', 'wikilink_18.json', 'wikilink_19.json', 'wikilink_2.json', 'wikilink_20.json', 'wikilink_21.json', 'wikilink_22.json', 'wikilink_23.json', 'wikilink_24.json', 'wikilink_25.json', 'wikilink_26.json', 'wikilink_27.json', 'wikilink_28.json', 'wikilink_29.json', 'wikilink_3.json', 'wikilink_30.json', 'wikilink_31.json', 'wikilink_32.json', 'wikilink_33.json', 'wikilink_34.json', 'wikilink_35.json', 'wikilink_36.json', 'wikilink_37.json', 'wikilink_38.json', 'wikilink_39.json', 'wikilink_4.json', 'wikilink_40.json', 'wikilink_41.json', 'wikilink_42.json', 'wikilink_43.json', 'wikilink_44.json', 'wikilink_45.json', 'wikilink_46.json', 'wikilink_47.json', 'wikilink_48.json', 'wikilink_49.json', 'wikilink_5.json', 'wikilink_50.json', 'wikilink_51.

In [139]:
df = pd.read_json(zf.open('wiki_test.json'))

In [135]:
df.head()

,wikis
0,{u'wikiurl': u'http://en.wikipedia.org/wiki/Ph...
1,{u'wikiurl': u'http://en.wikipedia.org/wiki/Wh...


In [62]:
[wiki['word'] for wiki in df.wikis]

[u'Pharoahe Monch', u'Whovian']

 ## Iterator code

##### in this example we will iterate and count all words in the data

simple list comprehend approach

In [117]:
zf.namelist()[4:6]

['wikilink_1.json', 'wikilink_10.json']

In [83]:
d = dict()
for link in zf.namelist()[4:6]: # deleta '6' to iterate over complete list
    print 'working on '+link
    df = pd.read_json(zf.open(link))
#     print df.size
    for key in df.wlinks:
        d[key['word']] = d.get(key['word'], 0) + 1

working on wikilink_1.json
working on wikilink_10.json


All items than appear more than 1 time

In [116]:
{k:v for k,v in d.iteritems() if v > 1}

{u'4+1 view model': 3,
 u'computer hacker': 3,
 u'Warcraft III: Reign of Chaos': 6,
 u'Piss Christ': 13,
 u'Allan Kardec': 2,
 u'Michel Geiss': 2,
 u'Fulton Fish Market': 2,
 u'John Henry': 14,
 u'Ferdinand "Butzi" Porsche': 2,
 u'Field of Glass': 2,
 u'hanging': 27,
 u'New York Court of Appeals': 4,
 u'Sullenberger': 2,
 u'Amundsen-Scott South Pole Station': 4,
 u'ethnic tensions': 2,
 u'text message': 12,
 u'film reel': 3,
 u'OPAC': 3,
 u'procainamide': 2,
 u'Dreyfus affair': 3,
 u'Edward Low': 3,
 u'Goten': 3,
 u"Ba'al": 7,
 u'Gerald Weinberg': 3,
 u'Newport bus station': 8,
 u'cycling': 15,
 u'what is now called the Durrell Wildlife Conservation Trust and the Jersey Zoo (now renamed Durrell Wildlife) on the Channel Island of Jersey in 1958': 2,
 u'Thomas Kyd': 2,
 u'Western': 44,
 u'Southwark Bridge': 4,
 u'1791': 9,
 u'Soccer': 3,
 u'-Majestic-': 2,
 u'chthonic': 5,
 u'Euro': 10,
 u'Lucha Libre': 13,
 u'Mission Inn': 2,
 u'Eightfold Path': 2,
 u'vehicular manslaughter': 2,
 u'rheu

word count using pythons MapReduce ;)

** edit - mrjob is line-based decoder ( in different from JSOn). tThe decoding must be changed in order ot work with JSON files.
see - https://github.com/Yelp/mrjob/issues/715

In [32]:
%%file titlecount.py
from collections import defaultdict
from mrjob.job import MRJob

class mrWordCount( MRJob ):
    def _init_(self, *args, **kwargs):
        super(mrWordCount,self)._init_(*args,**kwargs)
        self.localWordCount = defaultdict(int)
    
    def mapper(self, key, line):
        if False:
            yield
            for title in line:
                self.localWordCount[title] += 1
                
    def mapper_final(self):
        for (title,count) in self.localWordCount.iteritems():
            yield title, count
            
    def reducer(self,word, ocurrences):
        yield title, sum( ocurrences)
    
    if __name__ == '__main__':  
          mrWordCount.run()  

Overwriting titlecount.py


In [40]:
! python titlecount.py big10.txt

Traceback (most recent call last):
  File "titlecount.py", line 4, in <module>
    class mrWordCount( MRJob ):
  File "titlecount.py", line 23, in mrWordCount
    mrWordCount.run()  
NameError: name 'mrWordCount' is not defined


In [39]:
import titlecount
reload(titlecount)

mr_job = titlecount.mrWordCount(args=['big10.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for title in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print key, value

AttributeError: 'mrWordCount' object has no attribute 'localWordCount'

In [ ]:
wc.map

In [15]:
for link in zf.namelist()[0:2]: # change to complete list
    df = pd.read_json(zf.open(link))
#     print df.size
    [lambda x:d[x] += 1 for wiki in df.wikis]

SyntaxError: invalid syntax (<ipython-input-15-072d3381b013>, line 4)

simple working example

In [35]:
%%file wordcount.py
from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):

    def mapper(self, _, line):
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1

    def reducer(self, key, values):
        yield key, sum(values)
        
    if __name__ == '__main__':  
          MRWordFrequencyCount.run()  # where MRWordCounter is your job class

Writing wordcount.py


In [38]:
import wordcount
reload(wordcount)

mr_job = wordcount.MRWordFrequencyCount(args=['big10.txt'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print key, value

chars 16273254
lines 273792
words 2740116
